In [35]:
#필수 설치+드라이브 마운트
!pip -q install ultralytics==8.* opencv-python

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [36]:
#드라이브 zip 복사후 압축 해제
import os, zipfile, shutil

# (예시) 네 드라이브 안에 있는 zip 경로로 수정
ZIP_IN_DRIVE = "/content/drive/MyDrive/rsna-pneumonia-detector.v1i.yolov8.zip"

WORKDIR = "/content/dataset"
os.makedirs(WORKDIR, exist_ok=True)

zip_local = "/content/rsna.zip"
shutil.copy(ZIP_IN_DRIVE, zip_local)

with zipfile.ZipFile(zip_local, 'r') as z:
    z.extractall(WORKDIR)

print("Extracted to:", WORKDIR)
!ls -la /content/dataset | head -n 50

Extracted to: /content/dataset
total 32
drwxr-xr-x 5 root root 4096 Jan 15 04:40 .
drwxr-xr-x 1 root root 4096 Jan 15 06:14 ..
-rw-r--r-- 1 root root  301 Jan 15 06:17 data.yaml
-rw-r--r-- 1 root root  170 Jan 15 06:17 README.dataset.txt
-rw-r--r-- 1 root root 1141 Jan 15 06:17 README.roboflow.txt
drwxr-xr-x 4 root root 4096 Jan 15 04:40 test
drwxr-xr-x 4 root root 4096 Jan 15 04:41 train
drwxr-xr-x 4 root root 4096 Jan 15 04:41 valid


In [37]:
#yaml 위치
import glob

yamls = glob.glob("/content/dataset/**/*.yaml", recursive=True)
yamls[:20], len(yamls)

DATA_YAML = yamls[0]
print("Using:", DATA_YAML)

Using: /content/dataset/data.yaml


In [38]:
#yaml
import yaml, pprint

with open(DATA_YAML, "r") as f:
    cfg = yaml.safe_load(f)

In [42]:
#폐 포스팅
import os, glob, shutil
import cv2
import numpy as np

# --- 폐 마스킹 + 대비개선 함수(간단 버전) ---
def enhance_contrast_clahe(gray, clipLimit=2.0, tileGridSize=(8,8)):
    clahe = cv2.createCLAHE(clipLimit=clipLimit, tileGridSize=tileGridSize)
    return clahe.apply(gray)

def lung_mask_simple(gray):
    g = cv2.GaussianBlur(gray, (5,5), 0)
    _, th = cv2.threshold(g, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

    k = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (9,9))
    th = cv2.morphologyEx(th, cv2.MORPH_OPEN, k, iterations=1)
    th = cv2.morphologyEx(th, cv2.MORPH_CLOSE, k, iterations=2)

    n, labels, stats, _ = cv2.connectedComponentsWithStats(th, connectivity=8)
    mask = np.zeros_like(th)
    if n > 1:
        areas = stats[1:, cv2.CC_STAT_AREA]
        idx = np.argsort(areas)[::-1][:3]  # 큰 컴포넌트 2~3개 유지
        for kidx in idx:
            mask[labels == (kidx+1)] = 255

    mask = cv2.GaussianBlur(mask, (11,11), 0)
    mask = (mask > 50).astype(np.uint8) * 255
    return mask

def apply_lung_preproc(img_bgr):
    gray = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2GRAY)
    mask = lung_mask_simple(gray)
    masked = cv2.bitwise_and(gray, gray, mask=mask)
    enhanced = enhance_contrast_clahe(masked, clipLimit=2.0, tileGridSize=(8,8))
    return enhanced  # grayscale

# --- 원본/전처리 루트 ---
SRC_ROOT = "/content/dataset"
DST_ROOT = "/content/dataset_preproc"

# Roboflow 스타일(split/images, split/labels)이라고 가정
splits = ["train", "valid", "test"]

# 새 폴더 초기화
if os.path.exists(DST_ROOT):
    shutil.rmtree(DST_ROOT)
os.makedirs(DST_ROOT, exist_ok=True)

for sp in splits:
    src_img = os.path.join(SRC_ROOT, sp, "images")
    src_lbl = os.path.join(SRC_ROOT, sp, "labels")
    if not os.path.isdir(src_img):
        continue

    dst_img = os.path.join(DST_ROOT, sp, "images")
    dst_lbl = os.path.join(DST_ROOT, sp, "labels")
    os.makedirs(dst_img, exist_ok=True)
    os.makedirs(dst_lbl, exist_ok=True)

    # labels는 그대로 복사 (bbox는 그대로 유효)
    if os.path.isdir(src_lbl):
        shutil.copytree(src_lbl, dst_lbl, dirs_exist_ok=True)

    img_paths = sorted(glob.glob(src_img + "/*"))
    print(sp, "images:", len(img_paths))

    for p in img_paths:
        im = cv2.imread(p)
        if im is None:
            continue
        out = apply_lung_preproc(im)
        cv2.imwrite(os.path.join(dst_img, os.path.basename(p)), out)

print("Preprocessed dataset saved to:", DST_ROOT)

train images: 11796
valid images: 1140
test images: 558
Preprocessed dataset saved to: /content/dataset_preproc


In [49]:
#이미지개선
import os, glob, shutil
import cv2
import numpy as np

# (1) 전처리 함수들
def enhance_contrast_clahe(gray, clipLimit=2.0, tileGridSize=(8,8)):
    """명암/대비 개선(CLAHE): 의료영상에서 자주 사용"""
    clahe = cv2.createCLAHE(clipLimit=clipLimit, tileGridSize=tileGridSize)
    return clahe.apply(gray)

def gamma_correction(gray, gamma=1.2):
    """감마 보정: 1.0보다 크면 전체가 조금 밝아지는 경향(상황에 따라 조절)"""
    inv = 1.0 / gamma
    table = np.array([(i / 255.0) ** inv * 255 for i in range(256)]).astype("uint8")
    return cv2.LUT(gray, table)

def unsharp_mask(gray, sigma=1.0, strength=1.1):
    """선명도 개선(엣지 강화). 과하면 노이즈 증가"""
    blurred = cv2.GaussianBlur(gray, (0,0), sigmaX=sigma, sigmaY=sigma)
    sharp = cv2.addWeighted(gray, 1.0 + strength, blurred, -strength, 0)
    return np.clip(sharp, 0, 255).astype(np.uint8)

def lung_mask_simple(gray):
    """
    간단 폐 ROI 마스크(흉부 X-ray에 맞춘 휴리스틱)
    - 완벽한 분할은 아니지만 '배경 억제' 목적으론 쓸만함
    """
    g = cv2.GaussianBlur(gray, (5,5), 0)

    # Otsu + Invert (흉부에서 배경/기기 영역 제거용)
    _, th = cv2.threshold(g, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

    k = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (9,9))
    th = cv2.morphologyEx(th, cv2.MORPH_OPEN, k, iterations=1)
    th = cv2.morphologyEx(th, cv2.MORPH_CLOSE, k, iterations=2)

    n, labels, stats, _ = cv2.connectedComponentsWithStats(th, connectivity=8)
    mask = np.zeros_like(th)

    if n > 1:
        areas = stats[1:, cv2.CC_STAT_AREA]
        idx = np.argsort(areas)[::-1][:3]  # 큰 덩어리 2~3개 유지(좌/우 폐 대응)
        for kidx in idx:
            mask[labels == (kidx+1)] = 255

    # 경계 부드럽게
    mask = cv2.GaussianBlur(mask, (11,11), 0)
    mask = (mask > 50).astype(np.uint8) * 255
    return mask

def preprocess_xray(img_bgr,
                    use_clahe=True,
                    use_gamma=False,
                    gamma=1.2,
                    use_unsharp=True):
    """
    순서:
    1) grayscale
    2) lung ROI 마스크 적용(배경 억제)
    3) CLAHE로 대비 개선
    4) (선택) 감마
    5) (선택) unsharp
    """
    gray = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2GRAY)

    # (A) 폐 ROI 마스킹
    mask = lung_mask_simple(gray)
    roi = cv2.bitwise_and(gray, gray, mask=mask)

    # (B) 이미지 품질 개선
    out = roi
    if use_clahe:
        out = enhance_contrast_clahe(out, clipLimit=2.0, tileGridSize=(8,8))
    if use_gamma:
        out = gamma_correction(out, gamma=gamma)
    if use_unsharp:
        out = unsharp_mask(out, sigma=1.0, strength=1.1)

    return out  # grayscale (H,W)

# (2) 폴더 전체 전처리: images만 새로 만들고 labels는 복사
SRC_ROOT = "/content/dataset"
DST_ROOT = "/content/dataset_preproc"

splits = ["train", "valid", "test"]  #데이터 구조가 이 형태라면 그대로

if os.path.exists(DST_ROOT):
    shutil.rmtree(DST_ROOT)
os.makedirs(DST_ROOT, exist_ok=True)

for sp in splits:
    src_img_dir = os.path.join(SRC_ROOT, sp, "images")
    src_lbl_dir = os.path.join(SRC_ROOT, sp, "labels")

    if not os.path.isdir(src_img_dir):
        continue

    dst_img_dir = os.path.join(DST_ROOT, sp, "images")
    dst_lbl_dir = os.path.join(DST_ROOT, sp, "labels")
    os.makedirs(dst_img_dir, exist_ok=True)
    os.makedirs(dst_lbl_dir, exist_ok=True)

    # labels 복사 (bbox는 그대로 유효)
    if os.path.isdir(src_lbl_dir):
        shutil.copytree(src_lbl_dir, dst_lbl_dir, dirs_exist_ok=True)

    img_paths = sorted(glob.glob(src_img_dir + "/*"))
    print(f"[{sp}] images:", len(img_paths))

    for p in img_paths:
        img = cv2.imread(p)
        if img is None:
            continue
        out = preprocess_xray(img, use_clahe=True, use_gamma=False, use_unsharp=True)
        cv2.imwrite(os.path.join(dst_img_dir, os.path.basename(p)), out)

print(" Preprocessed dataset saved to:", DST_ROOT)

[train] images: 11796
[valid] images: 1140
[test] images: 558
 Preprocessed dataset saved to: /content/dataset_preproc


In [50]:
#학습
from ultralytics import YOLO

model = YOLO("yolov8s.pt")

results = model.train(
    data=DATA_YAML,
    epochs=15,
    imgsz=640,
    batch=16,
    device=0,
    workers=2,
    project="/content/runs",
    name="rsna_yolov8s_153",
    patience=0
)

Ultralytics 8.4.2 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (NVIDIA L4, 22693MiB)
engine/trainer: agnostic_nms=False, amp=True, angle=1.0, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/dataset/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=15, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8s.pt, momentum=0.937, mosaic=1.0, multi_scale=0.0, name=rsna_yolov8s_153, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=0, perspective=0.0, plots=

In [51]:
#저장 결과
import glob
from PIL import Image
import matplotlib.pyplot as plt

run_dir = "/content/runs/detect/rsna_yolov8s_153"
imgs = [
    "results.png",
    "confusion_matrix.png",
    "PR_curve.png",
    "F1_curve.png",
    "P_curve.png",
    "R_curve.png"
]

for fn in imgs:
    p = f"{run_dir}/{fn}"
    if os.path.exists(p):
        img = Image.open(p)
        plt.figure(figsize=(12,6))
        plt.imshow(img)
        plt.axis("off")
        plt.title(fn)
        plt.show()
    else:
        print("Not found:", p)

Not found: /content/runs/detect/rsna_yolov8s_153/results.png
Not found: /content/runs/detect/rsna_yolov8s_153/confusion_matrix.png
Not found: /content/runs/detect/rsna_yolov8s_153/PR_curve.png
Not found: /content/runs/detect/rsna_yolov8s_153/F1_curve.png
Not found: /content/runs/detect/rsna_yolov8s_153/P_curve.png
Not found: /content/runs/detect/rsna_yolov8s_153/R_curve.png


In [52]:
#시각화
import os, glob, cv2
import numpy as np
from ultralytics import YOLO

RUN_DIR  = "/content/runs/rsna_yolov8s_153"
BEST_PT  = f"{RUN_DIR}/weights/best.pt"
OUT_DIR  = "/content/gt_pred_overlay_s153"
os.makedirs(OUT_DIR, exist_ok=True)

# ---- (1) 이미지/라벨 폴더 자동 찾기 ----
def pick_dir(cands):
    cands = [p for p in cands if os.path.isdir(p)]
    scored = []
    for p in cands:
        n = len(glob.glob(os.path.join(p, "*")))
        scored.append((n, p))
    scored.sort(reverse=True)
    return scored[0][1] if scored and scored[0][0] > 0 else None, scored

img_cands = (
    glob.glob("/content/dataset/**/valid/images", recursive=True) +
    glob.glob("/content/dataset/**/val/images", recursive=True) +
    glob.glob("/content/dataset/**/images/val", recursive=True) +
    glob.glob("/content/dataset/**/test/images", recursive=True)
)

lbl_cands = (
    glob.glob("/content/dataset/**/valid/labels", recursive=True) +
    glob.glob("/content/dataset/**/val/labels", recursive=True) +
    glob.glob("/content/dataset/**/labels/val", recursive=True) +
    glob.glob("/content/dataset/**/test/labels", recursive=True)
)

IMG_DIR, img_scored = pick_dir(img_cands)
LBL_DIR, lbl_scored = pick_dir(lbl_cands)

print("IMG candidates (top):", img_scored[:5])
print("LBL candidates (top):", lbl_scored[:5])
print("Using IMG_DIR:", IMG_DIR)
print("Using LBL_DIR:", LBL_DIR)

assert IMG_DIR is not None, "이미지 폴더를 못 찾았어. dataset 구조를 다시 확인해야 함."
assert LBL_DIR is not None, "라벨 폴더를 못 찾았어. dataset 구조를 다시 확인해야 함."

# ---- (2) 유틸 ----
def yolo_to_xyxy(xc, yc, w, h, img_w, img_h):
    x1 = (xc - w/2) * img_w
    y1 = (yc - h/2) * img_h
    x2 = (xc + w/2) * img_w
    y2 = (yc + h/2) * img_h
    return np.array([x1, y1, x2, y2], dtype=np.float32)

def box_iou_xyxy(a, b):
    x1 = max(a[0], b[0]); y1 = max(a[1], b[1])
    x2 = min(a[2], b[2]); y2 = min(a[3], b[3])
    inter = max(0, x2-x1) * max(0, y2-y1)
    area_a = max(0, a[2]-a[0]) * max(0, a[3]-a[1])
    area_b = max(0, b[2]-b[0]) * max(0, b[3]-b[1])
    union = area_a + area_b - inter + 1e-9
    return inter / union

def read_yolo_labels(txt_path):
    if not os.path.exists(txt_path):
        return []
    items = []
    with open(txt_path, "r") as f:
        for line in f:
            line = line.strip()
            if not line:
                continue
            p = line.split()
            cls = int(float(p[0]))
            xc, yc, bw, bh = map(float, p[1:5])
            items.append((cls, xc, yc, bw, bh))
    return items

def draw_box(img, box_xyxy, color_bgr, text=None, thickness=2):
    x1,y1,x2,y2 = box_xyxy.astype(int)
    x1 = max(0, x1); y1 = max(0, y1)
    x2 = min(img.shape[1]-1, x2); y2 = min(img.shape[0]-1, y2)
    cv2.rectangle(img, (x1,y1), (x2,y2), color_bgr, thickness)
    if text:
        (tw, th), _ = cv2.getTextSize(text, cv2.FONT_HERSHEY_SIMPLEX, 0.6, 2)
        cv2.rectangle(img, (x1, max(0, y1-th-6)), (x1+tw+6, y1), color_bgr, -1)
        cv2.putText(img, text, (x1+3, y1-4),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255,255,255), 2, cv2.LINE_AA)

# ---- (3) 실행 ----
model = YOLO(BEST_PT)

img_paths = sorted(glob.glob(os.path.join(IMG_DIR, "*")))
print("num images:", len(img_paths))

# 너무 많으면 일부만(원하면 숫자 바꿔)
MAX_N = min(50, len(img_paths))

for img_path in img_paths[:MAX_N]:
    img = cv2.imread(img_path)
    if img is None:
        continue
    H, W = img.shape[:2]

    base = os.path.splitext(os.path.basename(img_path))[0]
    gt_txt = os.path.join(LBL_DIR, base + ".txt")

    # GT boxes (파랑)
    gt_list = read_yolo_labels(gt_txt)
    gt_boxes = [(cls, yolo_to_xyxy(xc,yc,bw,bh,W,H)) for cls,xc,yc,bw,bh in gt_list]

    # Pred boxes (빨강)
    r = model.predict(source=img_path, conf=0.25, verbose=False)[0]
    pred_boxes = []
    if r.boxes is not None and len(r.boxes) > 0:
        xyxy = r.boxes.xyxy.cpu().numpy().astype(np.float32)
        conf = r.boxes.conf.cpu().numpy()
        cls  = r.boxes.cls.cpu().numpy().astype(int)
        for c, b, s in zip(cls, xyxy, conf):
            pred_boxes.append((c, b, float(s)))

    # Pred 먼저 그림(빨강)
    for c, b, s in pred_boxes:
        draw_box(img, b, (0,0,255), text=f"Pred c{c} {s:.2f}", thickness=2)

    # GT 그림 + IoU 표시(파랑)
    for gt_c, gt_b in gt_boxes:
        best_iou = 0.0
        best = None
        for pc, pb, ps in pred_boxes:
            # 클래스까지 일치 매칭하려면 아래 주석 해제
            # if pc != gt_c:
            #     continue
            iou = box_iou_xyxy(gt_b, pb)
            if iou > best_iou:
                best_iou = iou
                best = (pc, pb, ps)

        draw_box(img, gt_b, (255,0,0), text=f"GT c{gt_c} IoU={best_iou:.3f}", thickness=2)

        # 매칭된 pred는 조금 두껍게 강조
        if best is not None and best_iou > 0:
            pc, pb, ps = best
            draw_box(img, pb, (0,0,255), text=f"Matched IoU={best_iou:.3f}", thickness=3)

    cv2.imwrite(os.path.join(OUT_DIR, base + "_overlay.jpg"), img)

print("Saved overlays to:", OUT_DIR)

IMG candidates (top): [(1140, '/content/dataset/valid/images'), (558, '/content/dataset/test/images')]
LBL candidates (top): [(1140, '/content/dataset/valid/labels'), (558, '/content/dataset/test/labels')]
Using IMG_DIR: /content/dataset/valid/images
Using LBL_DIR: /content/dataset/valid/labels
num images: 1140
Saved overlays to: /content/gt_pred_overlay_s153


In [47]:
#학습 완료했는지 확인
!find /content -maxdepth 6 -type f -name "results.csv" -o -name "results.png" | head -n 50

/content/runs/rsna_yolov8s_15/results.csv
/content/runs/rsna_yolov8s_15/results.png
/content/runs/rsna_yolov8n_15/results.csv
/content/runs/rsna_yolov8n_15/results.png
/content/runs/rsna_yolov8s_1522/results.csv
/content/runs/rsna_yolov8s_1522/results.png


In [53]:
#실험 결과도출
import os, glob
import pandas as pd
import numpy as np
from ultralytics import YOLO

RUNS = {
    "YOLOv8s": "/content/runs/rsna_yolov8s_153",  #경로
}

#(val 폴더는 너 캡처대로 이미 OK)
IMG_DIR = "/content/dataset/valid/images"
LBL_DIR = "/content/dataset/valid/labels"

def pick_col(df, keys):
    for k in keys:
        if k in df.columns:
            return k
    return None

def load_metrics_from_results_csv(run_dir):
    csv_path = os.path.join(run_dir, "results.csv")
    if not os.path.exists(csv_path):
        return None
    df = pd.read_csv(csv_path)
    last = df.iloc[-1]
    c_map50   = pick_col(df, ["metrics/mAP50(B)", "metrics/mAP50", "map50"])
    c_map     = pick_col(df, ["metrics/mAP50-95(B)", "metrics/mAP50-95", "map"])
    c_prec    = pick_col(df, ["metrics/precision(B)", "metrics/precision", "precision"])
    c_recall  = pick_col(df, ["metrics/recall(B)", "metrics/recall", "recall"])
    return {
        "mAP50": float(last[c_map50]) if c_map50 else np.nan,
        "mAP50-95": float(last[c_map]) if c_map else np.nan,
        "Precision": float(last[c_prec]) if c_prec else np.nan,
        "Recall": float(last[c_recall]) if c_recall else np.nan,
    }

def yolo_txt_to_xyxy(txt_path, W, H):
    import numpy as np
    boxes = []
    if not os.path.exists(txt_path):
        return boxes
    with open(txt_path, "r") as f:
        for line in f:
            p = line.strip().split()
            if len(p) < 5:
                continue
            cls = int(float(p[0]))
            xc, yc, bw, bh = map(float, p[1:5])
            x1 = (xc - bw/2)*W; y1 = (yc - bh/2)*H
            x2 = (xc + bw/2)*W; y2 = (yc + bh/2)*H
            boxes.append((cls, np.array([x1,y1,x2,y2], dtype=np.float32)))
    return boxes

def iou(a, b):
    x1 = max(a[0], b[0]); y1 = max(a[1], b[1])
    x2 = min(a[2], b[2]); y2 = min(a[3], b[3])
    inter = max(0, x2-x1) * max(0, y2-y1)
    area_a = max(0, a[2]-a[0]) * max(0, a[3]-a[1])
    area_b = max(0, b[2]-b[0]) * max(0, b[3]-b[1])
    return inter / (area_a + area_b - inter + 1e-9)

def compute_fp_fn(model, img_dir, lbl_dir, conf=0.25, iou_thr=0.5, match_class=False, max_images=None):
    import cv2
    img_paths = sorted(glob.glob(os.path.join(img_dir, "*")))
    if max_images:
        img_paths = img_paths[:max_images]
    FP = 0; FN = 0

    for img_path in img_paths:
        img = cv2.imread(img_path)
        if img is None:
            continue
        H, W = img.shape[:2]
        base = os.path.splitext(os.path.basename(img_path))[0]
        gt_path = os.path.join(lbl_dir, base + ".txt")

        gt = yolo_txt_to_xyxy(gt_path, W, H)
        gt_used = [False]*len(gt)

        r = model.predict(img_path, conf=conf, verbose=False)[0]
        preds = []
        if r.boxes is not None and len(r.boxes) > 0:
            xyxy = r.boxes.xyxy.cpu().numpy().astype(np.float32)
            cls  = r.boxes.cls.cpu().numpy().astype(int)
            for c, b in zip(cls, xyxy):
                preds.append((c, b))
        pred_used = [False]*len(preds)

        for pi, (pc, pb) in enumerate(preds):
            best_i = -1; best_v = 0.0
            for gi, (gc, gb) in enumerate(gt):
                if gt_used[gi]:
                    continue
                if match_class and pc != gc:
                    continue
                v = iou(pb, gb)
                if v > best_v:
                    best_v = v; best_i = gi
            if best_i >= 0 and best_v >= iou_thr:
                pred_used[pi] = True
                gt_used[best_i] = True

        FP += sum(1 for u in pred_used if not u)
        FN += sum(1 for u in gt_used if not u)

    return FP, FN

rows = []
for model_name, run_dir in RUNS.items():
    best_pt = os.path.join(run_dir, "weights", "best.pt")
    print(model_name, "best.pt:", best_pt, "exists?", os.path.exists(best_pt))

    if not os.path.exists(best_pt):
        continue

    m = load_metrics_from_results_csv(run_dir) or {"mAP50":np.nan,"mAP50-95":np.nan,"Precision":np.nan,"Recall":np.nan}
    model = YOLO(best_pt)
    FP, FN = compute_fp_fn(model, IMG_DIR, LBL_DIR, conf=0.25, iou_thr=0.5, match_class=False)

    rows.append({
        "모델 스케일": model_name,
        "mAP50": m["mAP50"],
        "mAP50-95": m["mAP50-95"],
        "Precision": m["Precision"],
        "Recall": m["Recall"],
        "FP (오탐)": FP,
        "FN (미탐)": FN,
        "총 에러 수": FP + FN
    })

df = pd.DataFrame(rows)

print("\n=== rows 개수 ===", len(rows))
print(df)              #무조건 텍스트로 출력돼서 “빈 표인지” 바로 확인 가능
df.style               #표 예쁘게 렌더링

YOLOv8s best.pt: /content/runs/rsna_yolov8s_153/weights/best.pt exists? True

=== rows 개수 === 1
    모델 스케일    mAP50  mAP50-95  Precision   Recall  FP (오탐)  FN (미탐)  총 에러 수
0  YOLOv8s  0.48973   0.17124    0.51445  0.58784      407     1069    1476


,모델 스케일,mAP50,mAP50-95,Precision,Recall,FP (오탐),FN (미탐),총 에러 수
0,YOLOv8s,0.489730,0.171240,0.514450,0.587840,407,1069,1476


In [54]:
#파일 크기/수정 시간 확인
!ls -lh /content/runs/rsna_yolov8s_153/weights/best.pt

-rw-r--r-- 1 root root 22M Jan 15 08:51 /content/runs/rsna_yolov8s_153/weights/best.pt
